In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
from os.path import join as pjoin

sys.path.append("../")
import circletrack_behavior as ctb

In [2]:
## Settings
dpath = '../../../../'
spath = './intermediate_data'
experimenters = ['Austin']
project_folders = ['CircleTrack_Inhibition', 'CircleTrack_Recall', 'MultiCon_Behavior']
experiment_list = ['Inhibition1', 
                   'Recall1', 
                   'Recall2', 
                   'Recall3', 
                   'Recall4',
                   'MultiCon_Lesion3']

if not os.path.exists(spath):
    os.makedirs(spath)

### Generate data frame of circle track rewards, first lick accuracy, fifth lick accuracy, etc.

In [3]:
circletrack_results = {'mouse': [], 'day': [], 'sex': [], 'age': [], 'experiment': [], 'session': [], 'experimenter': [], 'strain': [], 'maze': [],
                       'lick_accuracy': [], 'rewards': [], 'first_lick_accuracy': [], 'num_trials': [], 'percent_change': [], 'post_gain': []}

for experimenter in experimenters:
    if experimenter == 'Austin':
        expath = pjoin(dpath, f'{experimenter}/CircleTrack')
    for project in project_folders:
        proj_path = pjoin(expath, project)
        for experiment in os.listdir(proj_path):
            if experiment not in experiment_list:
                pass
            else:
                if experimenter == 'Austin':
                    circle_path = pjoin(proj_path, f'{experiment}/output/behav')
                    maze_info = pd.read_csv(pjoin(proj_path, f'{experiment}/maze_yml/maze_info.csv'))
                    mouse_info = pd.read_csv(pjoin(proj_path, f'{experiment}/maze_yml/mouse_info.csv'))
                    weight_info = pd.read_csv(pjoin(proj_path, f'{experiment}/maze_yml/weight_info.csv'))
                    ## Add day to weight_info for indexing later
                    num_mice = weight_info['mouse'].unique().shape[0]
                    weight_info['day'] = np.repeat([1, 2, 3, 4, 5], repeats=num_mice)
                else:
                    print('Currently waiting for BumJin...')
                    ## place-holder for future BumJin addition
                
                if (experiment == 'Recall1') | (experiment == 'Recall2') | (experiment == 'Recall3'):
                    experimenter_name = 'Naomi'
                elif (experiment == 'Recall4'):
                    experimenter_name = 'Ann' 
                else:
                    experimenter_name = experimenter

                for mouse in os.listdir(circle_path):
                    mouse_path = pjoin(circle_path, mouse)
                    sex = mouse_info['Sex'][mouse_info['Mouse'] == mouse].values[0]
                    exp = mouse_info['Experiment'][mouse_info['Mouse'] == mouse].values[0]
                    strain = mouse_info['Strain'][mouse_info['Mouse'] == mouse].values[0]
                    age = mouse_info['Age (months)'][mouse_info['Mouse'] == mouse].values[0]
                    for idx, session in enumerate(os.listdir(mouse_path)):
                        if idx > 4: ## only look at the first five days in A for all experiments, since this is consistent between experiments
                            pass 
                        else:
                            behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
                            behav = behav[~behav['probe']]
                            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                            pc_thresh1 = ctb.lick_accuracy(behav, port_list=[reward_one, reward_two], lick_threshold=1, by_trials=False)
                            pc_thresh5 = ctb.lick_accuracy(behav, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=False)
                            circletrack_results['mouse'].append(mouse)
                            circletrack_results['day'].append(idx+1)
                            circletrack_results['sex'].append(sex)
                            circletrack_results['age'].append(age)
                            circletrack_results['experiment'].append(exp)
                            circletrack_results['experimenter'].append(experimenter_name)
                            circletrack_results['strain'].append(strain)
                            circletrack_results['maze'].append(maze_info["if 'A'"][maze_info['Mouse'] == mouse].values[0])
                            circletrack_results['session'].append(np.unique(behav['session_two'])[0])
                            circletrack_results['lick_accuracy'].append(pc_thresh5)
                            circletrack_results['first_lick_accuracy'].append(pc_thresh1)
                            circletrack_results['rewards'].append(np.sum(behav['water']))
                            circletrack_results['num_trials'].append(behav['trials'].to_numpy()[-1])
                            circletrack_results['percent_change'].append(weight_info['percent_change'][(weight_info['mouse'] == mouse) & (weight_info['day'] == idx+1)].values[0])
                            circletrack_results['post_gain'].append(weight_info['post_gain'][(weight_info['mouse'] == mouse) & (weight_info['day'] == idx+1)].values[0])
ct_df = pd.DataFrame(circletrack_results)
ct_df.to_feather(pjoin(spath, 'ct_df.feat'))

### Generate probe performance data.

In [4]:
lick_dict_probe = {'mouse': [], 'day': [], 'sex': [], 'age': [], 'experiment': [], 'session': [], 'experimenter': [], 
                   'strain': [], 'maze': [], 'probe_acc': [], 'num_licks': []}

for experimenter in experimenters:
    if experimenter == 'Austin':
        expath = pjoin(dpath, f'{experimenter}/CircleTrack')
    for project in project_folders:
        proj_path = pjoin(expath, project)
        for experiment in os.listdir(proj_path):
            if experiment not in experiment_list:
                pass
            else:
                if experimenter == 'Austin':
                    circle_path = pjoin(proj_path, f'{experiment}/output/behav')
                    maze_info = pd.read_csv(pjoin(proj_path, f'{experiment}/maze_yml/maze_info.csv'))
                    mouse_info = pd.read_csv(pjoin(proj_path, f'{experiment}/maze_yml/mouse_info.csv'))
                    weight_info = pd.read_csv(pjoin(proj_path, f'{experiment}/maze_yml/weight_info.csv'))
                    ## Add day to weight_info for indexing later
                    num_mice = weight_info['mouse'].unique().shape[0]
                    weight_info['day'] = np.repeat([1, 2, 3, 4, 5], repeats=num_mice)
                else:
                    print('Currently waiting for BumJin...')
                    ## place-holder for future BumJin addition
                
                if (experiment == 'Recall1') | (experiment == 'Recall2') | (experiment == 'Recall3'):
                    experimenter_name = 'Naomi'
                elif (experiment == 'Recall4'):
                    experimenter_name = 'Ann' 
                else:
                    experimenter_name = experimenter

                for mouse in os.listdir(circle_path):
                    mouse_path = pjoin(circle_path, mouse)
                    sex = mouse_info['Sex'][mouse_info['Mouse'] == mouse].values[0]
                    exp = mouse_info['Experiment'][mouse_info['Mouse'] == mouse].values[0]
                    strain = mouse_info['Strain'][mouse_info['Mouse'] == mouse].values[0]
                    age = mouse_info['Age (months)'][mouse_info['Mouse'] == mouse].values[0]
                    for idx, session in enumerate(os.listdir(mouse_path)):
                        if idx > 4: ## only look at the first five days in A for all experiments, since this is consistent between experiments
                            pass 
                        else:
                            behav = pd.read_feather(pjoin(mouse_path, session))
                            if any(behav['probe']):
                                behav_probe = behav[behav['probe']]
                                behav_no_probe = behav[~behav['probe']]
                                reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
                                probe_acc = ctb.lick_accuracy(behav_probe, port_list=[reward_one, reward_two], lick_threshold=5, by_trials=False)
                                lick_dict_probe['mouse'].append(mouse)
                                lick_dict_probe['sex'].append(sex)
                                lick_dict_probe['age'].append(age)
                                lick_dict_probe['day'].append(idx+1)
                                lick_dict_probe['session'].append(np.unique(behav['session_two'])[0])
                                lick_dict_probe['experiment'].append(exp)
                                lick_dict_probe['experimenter'].append(experimenter_name)
                                lick_dict_probe['strain'].append(strain)
                                lick_dict_probe['maze'].append(maze_info["if 'A'"][maze_info['Mouse'] == mouse].values[0])
                                lick_dict_probe['num_licks'].append(len(behav_probe[behav_probe['lick_port'] != -1]))
                                lick_dict_probe['probe_acc'].append(probe_acc)
                            else:
                                pass
probe_df = pd.DataFrame(lick_dict_probe)
probe_df.to_feather(pjoin(spath, 'probe_df.feat'))